In [1]:
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp, Statevector
from tabulate import tabulate
from itertools import combinations
from qiskit.circuit.library import EfficientSU2
import math
import time
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.time_evolvers.variational import ImaginaryMcLachlanPrinciple
from qiskit.algorithms import TimeEvolutionProblem
from qiskit.algorithms import VarQITE
from qiskit.primitives import Estimator
from qiskit.quantum_info import Statevector
from qiskit.algorithms import SciPyImaginaryEvolver
import pylab
from qiskit.algorithms.gradients import ReverseEstimatorGradient, ReverseQGT
from qiskit.circuit.library import RYGate
import matplotlib.pyplot as plt

import os
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
'''Check the hamiltonian, changed the parameters for this file'''
def tb0_JW(N,e,t) : 
    strings = []
    opt = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits=N)  
    for k in range(N) : 
        a0='I'*(N)
        a1 = 'I'*(k)+'Z' +'I'*(N-k-1)

        b0='I'*N
        b0_list = list(b0)
        b0_list[k] = 'X'
        b0_list[(k+1)%N] = 'X'
        new_b0 = ''.join(b0_list)

        b1='I'*N
        b1_list = list(b0)
        b1_list[k] = 'Y'
        b1_list[(k+1)%N] = 'Y'
        new_b1 = ''.join(b1_list)

        strings.append([a0,a1,new_b0,new_b1])
        val = 1
        if N==2 and k==1 : 
            val = 0
        opt += SparsePauliOp.from_list([(a0, 0.5*e[k]), (a1, -0.5*e[k]),(new_b0, 0.5*t*val),(new_b1, 0.5*t*val)])
    return opt  

def spinless_basis(N,r) : 
    basis_set = []
    lattice = list(range(N))
    places = list(combinations(lattice, r))
    for combination in places : 
        basis = [False] *N
        for index in combination : 
            basis[index] = True 
        basis_set.append(basis)
    return basis_set

def scs_param(n,l,var,param) : 

    circ = QuantumCircuit(n)
    circ.cx(-2,-1)
    circ.cry(param[var],-1,-2)
    var+=1
    circ.cx(-2,-1)

    for i in range(l-1) : 
        circ.cx(-3-i,-1)
        ccry = RYGate(param[var]).control(2,label=None)
        var+=1
        circ.append(ccry,[-1,-2-i,-3-i])
        circ.cx(-3-i,-1)
    return circ, var

def dicke_param(n,k) : 
    pairs = []
    for a in range(n,k,-1) : 
        pairs.append([a,k])
    for a in range(k,1,-1) : 
        pairs.append([a,a-1])

    num_angles = int(k*(n-k) + k*(k-1)/2)+k
    param = [Parameter(f"angle_{i+1}") for i in range(num_angles)]

    dk_circ = QuantumCircuit(n)
    for q in range(k) : 
        dk_circ.x(-(q+1))
    var=0

    for pair in pairs : 
        new_circ,new_var = scs_param(pair[0],pair[1],var,param)
        var = new_var
        dk_circ.append(new_circ, range(pair[0]))
    return dk_circ




In [3]:
backend = Aer.get_backend('statevector_simulator')

In [6]:
# folder_name = "Dicke plots"
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)

pdf_filename = os.path.join("su2vsDicke_plots.pdf")
pdf_pages = PdfPages(pdf_filename)
results = []

for N in [3,4,6,7,9,10] : 
    #Defining Hamiltonian
    e=[1]*N
    t=1
    H_op = tb0_JW(N,e,t)
    
    #Classically finding eigvectors and eigenvalues
    eig,vec = np.linalg.eig(H_op.to_matrix())
    new_vec = list(zip(*vec))

    eigstates = []
    for i,x in enumerate(eig) : 
        if np.round(x) == np.round(min(eig)) : 
            eigstates.append(new_vec[i])
    
    #This finds the right results for less number of qubits and takes a lot of time 
    su2 = EfficientSU2(H_op.num_qubits, reps=3)
    #su2.decompose().draw('mpl')
    #print(su2.parameters)
    
    #Deciding ansatz   
    ansatz_list = [su2, dicke_param(N,N//2)]
    ansatz_string = ['su2','dicke']
    for ind, ansatz in enumerate(ansatz_list) : 
        
        init_param_values={}
        for i in range(len(ansatz.parameters)):
            init_param_values[ansatz.parameters[i]]=1

        exp_time = 8.0
        num_steps = 200

        aux_ops = [H_op]
        evolution_problem = TimeEvolutionProblem(H_op, exp_time, aux_operators=aux_ops)

        start_time = time.time()

        #var_principle = ImaginaryMcLachlanPrinciple()
        var_principle = ImaginaryMcLachlanPrinciple(qgt = ReverseQGT() , 
                                                    gradient = ReverseEstimatorGradient())

        evolution_problem = TimeEvolutionProblem(H_op, exp_time, aux_operators=aux_ops)
        var_qite = VarQITE(ansatz, init_param_values, var_principle, Estimator(),
                          num_timesteps=num_steps)
        evolution_result_eff = var_qite.evolve(evolution_problem)
        end_time = time.time()
        elapsed_time = end_time - start_time

        eff_circ = evolution_result_eff.evolved_state

        eff_job = execute(eff_circ, backend)
        eff_result = eff_job.result()
        eff_statevector = eff_result.get_statevector()

        sum_of_squares = (np.array(eff_statevector).conj() @ np.array(eff_statevector)).real
        norm_state = eff_statevector/np.sqrt(sum_of_squares)
        final_sv = [np.round(x,3) for x in np.asarray(norm_state)]

        overlap_list =[]
        for state in eigstates : 
            overlap_list.append(np.dot(state,np.conj(norm_state)))

        h_exp_val = np.array([ele[0][0] for ele in evolution_result_eff.observables])

        #Classical simulation
        init_state = Statevector(ansatz.assign_parameters(init_param_values))

        evolution_problem = TimeEvolutionProblem(H_op, exp_time, initial_state=init_state, 
                                                 aux_operators=aux_ops)
        exact_evol = SciPyImaginaryEvolver(num_timesteps=num_steps)
        sol = exact_evol.evolve(evolution_problem)

        exact_h_exp_val = sol.observables[0][0].real
        #print("Exact lowest eigenvalue found : ",exact_h_exp_val[-1]) 

        times_eff = evolution_result_eff.times
        times_exact = sol.times
        
        plt.axhline(y=min(eig), color='black', linestyle='--', linewidth=1, alpha=0.5, 
            label=f'Exact eig = {np.round(min(eig),3)}')
        pylab.plot(times_eff, h_exp_val, label= "VarQITE")
        pylab.plot(times_exact, exact_h_exp_val , label= "Exact",  linestyle='--')
        pylab.xlabel("Time")
        pylab.ylabel(r"$\langle H \rangle$ (energy)")
        pylab.legend(loc="upper right");
        plt.title(f'N= {N}, steps = {num_steps}, {ansatz_string[ind]}')

        pdf_pages.savefig()
        plt.close()
        error = np.round(np.abs(min(eig).real-h_exp_val[-1].real),3)
        results.append({'Ansatz': ansatz_string[ind],'N' : N, 'error':error, 'TrueEig' : np.round(min(eig),3).real, 
                        'est_eig' : np.round(h_exp_val[-1],3), 'time' : np.round(elapsed_time,3), 
                        'StateAccuracy' : np.round(overlap_list[0],3), 'ClsclEig': np.round(exact_h_exp_val[-1],3) } )
    
pdf_pages.close()
df = pd.DataFrame(results)   
df.reset_index(drop=True, inplace=True)

/opt/anaconda3/lib/python3.9/site-packages/matplotlib/cbook/__init__.py:1298: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [7]:
print(df)
df.to_csv('su2vsDicke data.csv', index=False)

   Ansatz   N  error  TrueEig  est_eig     time       StateAccuracy  ClsclEig
0     su2   3  0.000   -0.000    0.000   62.388 -0.089000+0.416000j     0.000
1   dicke   3  0.000   -0.000    0.000    3.919 -0.651000+0.000000j     0.000
2     su2   4  0.175   -1.000   -0.825  109.635 -0.599000+0.624000j    -0.995
3   dicke   4  0.268   -1.000   -0.732   21.246  0.000000+0.000000j    -0.828
4     su2   6  0.417   -1.464   -1.048  254.293  0.570000-0.458000j    -1.464
5   dicke   6  0.677   -1.464   -0.787  114.500  0.000000+0.000000j    -1.000
6     su2   7  0.349   -1.247   -0.898  344.625 -0.157000-0.320000j    -1.238
7   dicke   7  0.357   -1.247   -0.890  175.561 -0.000000-0.000000j    -1.049
8     su2   9  0.436   -1.759   -1.323  584.483  0.484000+0.463000j    -1.732
9   dicke   9  0.604   -1.759   -1.155  517.672  0.000000+0.000000j    -1.411
10    su2  10  0.533   -2.236   -1.703  733.995  0.396000-0.407000j    -2.207
11  dicke  10  1.291   -2.236   -0.945  983.109  0.000000+0.0000